In [1]:
import gan_protein_structural_requirements.utils as utils
from gan_protein_structural_requirements.utils.protein_visualizer import jupy_viz_obj as jvo, jupy_viz_file as jvf
from gan_protein_structural_requirements.utils.folding_models import esm_batch_predict as esm_batch_predict, esm_predict as esm_predict, load_tokenizer as load_tokenizer
from gan_protein_structural_requirements.utils.extract_structure import extract_structures as extract_structures, untokenize as untokenize, get_untokenizer as get_untokenizer

[11/19/23 15:27:19] WARNING  To use the Graphein submodule                                         ]8;id=522461;file:///Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=4954;file:///Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/graphein/protein/features/sequence/embeddings.py#44\44]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

[11/19/23 15:27:20] WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=841360;file:///Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=545171;file:///Users/rohitkulkarni/opt/anaconda3/envs/protein_proj/lib/python3.8/site-packages/graphein/protein/visualisation.py#35\35]8;;\
                             need to install: pytorch3d                                                            
                             To do so, use the following command: conda install -c pytorch3d                       
                             pytorch3d                                                                             

In [ ]:
data = extract_structures("./gan_protein_structural_requirements/data/raw/batch_1_data/")

In [ ]:
import pandas as pd
import numpy as np

arr = []

for entry in data["secondary"]:
    id = entry
    seq = data["secondary"][id]['sequence']
    c8 = data["secondary"][id]['c8']
    pol = data["primary_pol"][id]["polarity_conv"]
    arr.append([id, seq, c8, pol])

dat = np.array(arr)

df = pd.DataFrame(dat, columns=["id","seq","c8","polarity"])
df.head()

print(df.seq.map(lambda x: len(x)).max())

#max len of tokens = around 800-1000